In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("./datasets/movies.csv")
ratings = pd.read_csv("./datasets/ratings.csv")
tags = pd.read_csv("./datasets/tags.csv")

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [5]:
df = pd.merge(ratings, movies, on = "movieId", how = "left")
df = df.drop('title', axis = 1)
df.head()

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Comedy|Romance
2,1,6,4.0,964982224,Action|Crime|Thriller
3,1,47,5.0,964983815,Mystery|Thriller
4,1,50,5.0,964982931,Crime|Mystery|Thriller


In [6]:
df['genres'] = df['genres'].str.split("|")

In [7]:
df.drop('timestamp', axis = 1, inplace = True)
df.head()

,userId,movieId,rating,genres
0,1,1,4.0,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1,3,4.0,"[Comedy, Romance]"
2,1,6,4.0,"[Action, Crime, Thriller]"
3,1,47,5.0,"[Mystery, Thriller]"
4,1,50,5.0,"[Crime, Mystery, Thriller]"


In [8]:
tags.drop('timestamp', axis=1, inplace=True)
tags.drop('tag', axis = 1, inplace = True)

In [9]:
tags.head()

,userId,movieId
0,2,60756
1,2,60756
2,2,60756
3,2,89774
4,2,89774


In [10]:
df = pd.merge(df, tags, on=['userId','movieId'], how='left')

In [11]:
df.shape

(102677, 4)

In [13]:
df['genres'] = df['genres'].apply(lambda d: d if isinstance(d, list) else [])

In [14]:
df.head()

,userId,movieId,rating,genres
0,1,1,4.0,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1,3,4.0,"[Comedy, Romance]"
2,1,6,4.0,"[Action, Crime, Thriller]"
3,1,47,5.0,"[Mystery, Thriller]"
4,1,50,5.0,"[Crime, Mystery, Thriller]"


In [15]:
#boyut azaltma

min_ratings = 5
filter_items = df['movieId'].value_counts() > min_ratings
filter_items = filter_items[filter_items].index.tolist()

min_user_ratings = 5
filter_users = df['userId'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['movieId'].isin(filter_items)) & (df['userId'].isin(filter_users))]

print('Original shape {}'.format(df.shape))
print('New shape {}'.format(df_new.shape))

Original shape (102677, 4)
New shape (90213, 4)


In [16]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_new, test_size=0.2, stratify=df_new.userId)

In [17]:
train_data = train_data.sort_values(['userId', 'movieId'])
train_data.tail()

,userId,movieId,rating,genres
102670,610,164179,5.0,[Sci-Fi]
102672,610,166534,4.0,"[Drama, Horror, Thriller]"
102673,610,168248,5.0,"[Action, Crime, Thriller]"
102674,610,168250,5.0,[Horror]
102675,610,168252,5.0,"[Action, Sci-Fi]"


In [18]:
test_data = test_data.sort_values(['userId','movieId'])
test_data.tail()

,userId,movieId,rating,genres
102621,610,139385,4.5,"[Adventure, Drama]"
102633,610,142488,3.5,[Thriller]
102660,610,160080,3.0,"[Action, Comedy, Horror, Sci-Fi]"
102667,610,162350,3.5,"[Action, Western]"
102671,610,166528,4.0,"[Action, Adventure, Fantasy, Sci-Fi]"


In [19]:
train_data.to_csv('./datasets/training_data.csv', index = False)
test_data.to_csv('./datasets/testing_data.csv', index = False)

In [20]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].apply(lambda d: d if isinstance(d, list) else [])
movies.head()
movies.to_csv('./datasets/movies.csv', index = False)